# LangGraph Tools: EMI Calculator

## Objective
Build an EMI (Equated Monthly Installment) calculator tool that demonstrates complex financial calculations with multiple parameter types.

## What You'll Learn
1. Creating tools with mixed parameter types (float, int, str)
2. Implementing mathematical formulas in tools
3. Handling multiple validation scenarios
4. Formatting financial output for readability
5. Managing edge cases (zero interest rate)

## Prerequisites
- Completed: `02_getting_started_with_langgraph_tools.ipynb`
- Completed: `03_langgraph_tools_creation_currency_calculator.ipynb`
- Basic understanding of loan terminology (principal, interest, tenure)

---

## Section 1: Setup

Import the required dependencies.

In [ ]:
# Core imports
from langchain_core.tools import tool
from pprint import pprint

print("✅ Imports successful")

---

## Section 2: Understanding EMI Calculations

### What is EMI?
**EMI (Equated Monthly Installment)** is the fixed payment amount a borrower makes to a lender each month to repay a loan.

### The EMI Formula

```
EMI = P × r × (1 + r)^n / [(1 + r)^n - 1]
```

Where:
- **P** = Principal (loan amount)
- **r** = Monthly interest rate (annual rate ÷ 12 ÷ 100)
- **n** = Tenure in months

### Reference Point: Formula Breakdown

| Component | Formula | Example (₹5,00,000 at 8.5% for 5 years) |
|-----------|---------|----------------------------------------|
| Principal (P) | Given | ₹5,00,000 |
| Monthly Rate (r) | 8.5 ÷ 12 ÷ 100 | 0.007083 |
| Tenure (n) | 5 × 12 | 60 months |
| (1 + r)^n | (1.007083)^60 | 1.5257 |
| **EMI** | P × r × (1+r)^n / [(1+r)^n - 1] | **₹10,238.78** |

### Reference Point: Related Calculations

From EMI, we can derive:

```
Total Payment   = EMI × n
Total Interest  = Total Payment - Principal
```

These help users understand the true cost of borrowing.

---

## Section 3: Building the EMI Calculator Tool

Notice these key elements:
1. **Four parameters** with mixed types (`float`, `float`, `int`, `str`)
2. **Multiple validations** for different error conditions
3. **Edge case handling** for zero interest rate
4. **Rich formatted output** with multiple calculated values

In [ ]:
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment, 
    total repayment amount, or total interest for a loan.
    
    Args:
        principal: The loan amount (must be greater than 0)
        annual_interest_rate: Annual interest rate as percentage (e.g., 8.5 for 8.5%)
        tenure_months: Loan tenure in months (must be greater than 0)
        currency: Currency code for display (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with EMI calculation details including monthly payment,
        total payment, and total interest
    """
    
    # ===== INPUT VALIDATION =====
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    # ===== CALCULATION =====
    # Convert annual rate to monthly rate (decimal)
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    # Handle zero interest rate (edge case)
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        # EMI formula: P × r × (1 + r)^n / [(1 + r)^n - 1]
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    # ===== FORMAT OUTPUT =====
    # Convert months to years and remaining months for display
    years = tenure_months // 12
    remaining_months = tenure_months % 12
    
    result = (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months ({years} years, {remaining_months} months)\n"
        f"  \n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )
    
    return result

print("✅ emi_calculator tool defined")
print(f"   Tool Name: {emi_calculator.name}")
print(f"   Tool Type: {type(emi_calculator)}")

### Reference Point: Code Structure Pattern

Notice how the tool is organized into clear sections:

```python
@tool
def my_tool(params) -> str:
    """Docstring"""
    
    # ===== INPUT VALIDATION =====
    # Check all inputs before processing
    
    # ===== CALCULATION =====
    # Core business logic
    
    # ===== FORMAT OUTPUT =====
    # Prepare human-readable result
    
    return result
```

This pattern makes tools easier to read, debug, and maintain.

---

## Section 4: Testing the EMI Calculator

We'll test various scenarios:
1. Car loan (short-term, moderate amount)
2. Home loan (long-term, large amount)
3. Zero interest (edge case)
4. Invalid input (error handling)

In [ ]:
print("=" * 70)
print("TEST CASE 1: Car Loan")
print("Scenario: ₹5,00,000 at 8.5% for 5 years")
print("=" * 70)

result = emi_calculator.invoke({
    "principal": 500000,
    "annual_interest_rate": 8.5,
    "tenure_months": 60,
    "currency": "INR"
})
print(result)

In [ ]:
print("=" * 70)
print("TEST CASE 2: Home Loan")
print("Scenario: $300,000 at 6.5% for 30 years")
print("=" * 70)

result = emi_calculator.invoke({
    "principal": 300000,
    "annual_interest_rate": 6.5,
    "tenure_months": 360,
    "currency": "USD"
})
print(result)

### Reference Point: Interpreting Home Loan Results

For the $300,000 home loan above, notice:
- **Monthly EMI ≈ $1,896** - The fixed monthly payment
- **Total Payment ≈ $682,632** - What you actually pay over 30 years
- **Total Interest ≈ $382,632** - The cost of borrowing (more than the loan itself!)

> **Insight:** On a 30-year mortgage, you often pay more in interest than the original loan amount. This is valuable information for users making financial decisions.

In [ ]:
print("=" * 70)
print("TEST CASE 3: Zero Interest Loan (Edge Case)")
print("Scenario: €10,000 at 0% for 12 months")
print("=" * 70)

result = emi_calculator.invoke({
    "principal": 10000,
    "annual_interest_rate": 0,
    "tenure_months": 12,
    "currency": "EUR"
})
print(result)

### Reference Point: Why Handle Zero Interest?

The standard EMI formula fails when `r = 0` because:
- `(1 + 0)^n = 1`
- Denominator becomes `1 - 1 = 0`
- Division by zero!

**Solution:** Special case where `EMI = Principal ÷ Tenure`

This handles promotional "0% financing" offers that users might ask about.

In [ ]:
print("=" * 70)
print("TEST CASE 4: Invalid Principal (Error Handling)")
print("Scenario: Principal = 0")
print("=" * 70)

result = emi_calculator.invoke({
    "principal": 0,
    "annual_interest_rate": 8.5,
    "tenure_months": 24,
    "currency": "USD"
})
print(result)

In [ ]:
print("=" * 70)
print("TEST CASE 5: Negative Interest Rate (Error Handling)")
print("Scenario: Interest rate = -5%")
print("=" * 70)

result = emi_calculator.invoke({
    "principal": 10000,
    "annual_interest_rate": -5,
    "tenure_months": 12,
    "currency": "USD"
})
print(result)

print("\n" + "=" * 70)
print("✅ All tests completed")
print("=" * 70)

### Reference Point: Validation Strategy

| Parameter | Validation Rule | Rationale |
|-----------|----------------|----------|
| `principal` | Must be > 0 | Can't have a loan of zero or negative amount |
| `annual_interest_rate` | Must be ≥ 0 | Negative rates don't make sense; 0% is valid |
| `tenure_months` | Must be > 0 | Loan must have at least 1 month tenure |
| `currency` | No validation | Display only; doesn't affect calculation |

---

## Section 5: Examining the Tool Schema

Let's see how the four-parameter schema looks compared to the currency converter's three-parameter schema.

In [ ]:
print("EMI Calculator - Tool Schema")
print("=" * 70)
print("\nThis is what the LLM sees:\n")

schema = emi_calculator.args_schema.model_json_schema()
pprint(schema)

### Reference Point: Schema Type Mapping

Notice how Python types map to JSON Schema types:

| Parameter | Python Type | JSON Schema Type |
|-----------|-------------|------------------|
| `principal` | `float` | `number` |
| `annual_interest_rate` | `float` | `number` |
| `tenure_months` | `int` | `integer` |
| `currency` | `str` | `string` |

> **Note:** Both `float` and `int` are numeric, but `int` becomes `integer` (whole numbers only) while `float` becomes `number` (allows decimals). This helps the LLM provide appropriate values.

---

## Section 6: Real-World Usage Scenarios

Here's how users might interact with this tool through an LLM:

### Scenario 1: Direct Question
```
User: "What's the EMI for a ₹10 lakh home loan at 9% for 20 years?"

LLM extracts:
  - principal: 1000000 (10 lakh = 10,00,000)
  - annual_interest_rate: 9
  - tenure_months: 240 (20 × 12)
  - currency: "INR"
```

### Scenario 2: Comparison Request
```
User: "Compare EMI for $50,000 car loan at 7% for 3 years vs 5 years"

LLM makes TWO tool calls:
  Call 1: principal=50000, rate=7, tenure=36, currency="USD"
  Call 2: principal=50000, rate=7, tenure=60, currency="USD"
```

### Scenario 3: Incomplete Information
```
User: "Calculate EMI for my loan"

LLM response: "I'd be happy to calculate your EMI. Could you please provide:
  - Loan amount
  - Interest rate
  - Loan tenure (in months or years)
  - Currency?"
```

> **Key Insight:** The LLM uses the schema to know what parameters are required and will ask for missing information before calling the tool.

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **Mixed parameter types** | Use appropriate types (float for money, int for counts) |
| **Mathematical formulas** | Implement standard formulas with proper variable naming |
| **Edge case handling** | Special cases (like 0% interest) need explicit handling |
| **Multiple validations** | Validate each input with clear error messages |
| **Code organization** | Use sections: Validation → Calculation → Format Output |

## Series Complete!

You've now learned the fundamentals of creating LangGraph tools:

1. **Notebook 02:** Basic tool creation, `@tool` decorator, `.invoke()`, schemas
2. **Notebook 03:** Multi-parameter tools, validation, currency converter
3. **Notebook 04:** Complex calculations, mixed types, EMI calculator

### Next Steps
- Integrate these tools into a LangGraph workflow
- Connect tools to an LLM using `ToolNode`
- Build an agent that can use multiple tools

---

## Practice Exercises

Try extending the EMI calculator with these challenges:

1. **Add processing fee** - Include a one-time processing fee (usually 1-2% of principal) in the total cost
2. **Create a loan comparison tool** - Takes two interest rates and returns which is better
3. **Build a reverse EMI tool** - Given a desired EMI amount, calculate the maximum loan you can afford

In [ ]:
# Exercise 1: Add processing fee calculation
# Hint: Add a processing_fee_percent parameter (default 1%)
# Your code here:



In [ ]:
# Exercise 2: Create a loan comparison tool
# Input: principal, tenure, rate1, rate2, currency
# Output: Compare EMI, total payment, and total interest for both rates
# Your code here:



In [ ]:
# Exercise 3: Build a reverse EMI calculator (affordability calculator)
# Input: desired_emi, annual_interest_rate, tenure_months, currency
# Output: Maximum loan amount you can afford
# Hint: Rearrange the EMI formula to solve for P
# Your code here:

